# PGS s3e25 Ensemble w/VotingRegressor

https://www.kaggle.com/code/stpeteishii/pgs-s3e25-ensemble-w-optuna<br/>
https://www.kaggle.com/code/stpeteishii/pgs-s3e25-ensemble-w-stackingregressor

A VotingRegressor is a type of ensemble learning algorithm that combines the predictions of multiple base estimators to make a final prediction. Each base estimator is trained on the entire dataset, and the predictions of all the base estimators are averaged to produce the final prediction. The VotingRegressor can be used for both regression and classification tasks.

In [1]:
# Misc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os
from copy import deepcopy
from functools import partial
import gc
import warnings

# Import sklearn classes for model selection, cross validation, and performance evaluation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.metrics import roc_auc_score, log_loss, f1_score
from sklearn.metrics import precision_score, recall_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.preprocessing import scale
from category_encoders import OneHotEncoder, OrdinalEncoder, CountEncoder
from imblearn.under_sampling import RandomUnderSampler

# Import libraries for gradient boosting
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoost, CatBoostRegressor, CatBoostClassifier, Pool
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.svm import NuSVC, SVC
from sklearn.gaussian_process.kernels import RBF

from math import sqrt
from scipy.optimize import minimize

from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.ensemble import StackingRegressor, VotingRegressor

ModuleNotFoundError: No module named 'category_encoders'

In [2]:
# Seaborn
rc = {
    "axes.facecolor": "#FAEEE9",
    "figure.facecolor": "#FAEEE9",
    "axes.edgecolor": "#000000",
    "grid.color": "#EBEBE7",
    "font.family": "arial",
    "axes.labelcolor": "#000000",
    "xtick.color": "#000000",
    "ytick.color": "#000000",
    "grid.alpha": 0.4
}
sns.set(rc=rc)

# Useful line of code to set the display option so we could see all the columns in pd dataframe
pd.set_option('display.max_columns', None)

# Suppress warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

def print_sl():
    print("=" * 50)
    print()


In [4]:
train0 = pd.read_csv("/home/rotse4/Desktop/ml/train.csv")
test0 = pd.read_csv("/home/rotse4/Desktop/ml/test.csv")
display(train0[0:2].T)
data0=pd.concat([train0,test0],axis=0)
display(data0)
display(data0.info())

,0,1
id,0.000000,1.000000
allelectrons_Total,100.000000,100.000000
density_Total,0.841611,7.558488
allelectrons_Average,10.000000,10.000000
val_e_Average,4.800000,4.800000
atomicweight_Average,20.612526,20.298893
ionenergy_Average,11.088100,12.040830
el_neg_chi_Average,2.766000,2.755000
R_vdw_element_Average,1.732000,1.631000
R_cov_element_Average,0.860000,0.910000


,id,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average,Hardness
0,0,100.0,0.841611,10.000000,4.800000,20.612526,11.088100,2.766000,1.732000,0.860000,0.496070,0.91457,6.0
1,1,100.0,7.558488,10.000000,4.800000,20.298893,12.040830,2.755000,1.631000,0.910000,0.492719,0.71760,6.5
2,2,76.0,8.885992,15.600000,5.600000,33.739258,12.086300,2.828000,1.788000,0.864000,0.481478,1.50633,2.5
3,3,100.0,8.795296,10.000000,4.800000,20.213349,10.948500,2.648000,1.626000,0.936000,0.489272,0.78937,6.0
4,4,116.0,9.577996,11.600000,4.800000,24.988133,11.824480,2.766000,1.682000,0.896000,0.492736,1.86481,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6934,17341,446.0,36.135392,9.636364,4.636364,19.689448,11.045515,2.768636,1.769697,0.871818,0.498702,0.11548,NaN
6935,17342,36.0,3.550000,18.000000,4.000000,36.068500,8.236600,1.790000,2.055000,1.390000,0.499000,1.35045,NaN
6936,17343,68.0,4.545328,11.333333,5.333333,22.688853,10.938358,2.876667,1.713333,0.916667,0.499074,0.73425,NaN
6937,17344,40.0,2.334164,6.666667,4.000000,13.016128,12.700467,2.770000,1.476667,0.616667,0.663797,0.51227,NaN


<class 'pandas.core.frame.DataFrame'>
Index: 17346 entries, 0 to 6938
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     17346 non-null  int64  
 1   allelectrons_Total     17346 non-null  float64
 2   density_Total          17346 non-null  float64
 3   allelectrons_Average   17346 non-null  float64
 4   val_e_Average          17346 non-null  float64
 5   atomicweight_Average   17346 non-null  float64
 6   ionenergy_Average      17346 non-null  float64
 7   el_neg_chi_Average     17346 non-null  float64
 8   R_vdw_element_Average  17346 non-null  float64
 9   R_cov_element_Average  17346 non-null  float64
 10  zaratio_Average        17346 non-null  float64
 11  density_Average        17346 non-null  float64
 12  Hardness               10407 non-null  float64
dtypes: float64(12), int64(1)
memory usage: 1.9 MB


None

In [5]:
from sklearn.preprocessing import LabelEncoder

def labelencoder(df):
    for c in df.columns:
        if df[c].dtype=='object': 
            df[c] = df[c].fillna('N')
            lbl = LabelEncoder()
            lbl.fit(list(df[c].values))
            df[c] = lbl.transform(df[c].values)
    return df

In [6]:
data=labelencoder(data0)
train=data.iloc[0:len(train0)]
test=data.iloc[len(train0):]

In [7]:
target=['Hardness']
trainY=train['Hardness']
trainX0=train.drop(['Hardness','id'],axis=1)
testX0=test.drop(['Hardness','id'],axis=1)
cols=testX0.columns.tolist()
print(cols)
display(trainX0[0:2].T)

['allelectrons_Total', 'density_Total', 'allelectrons_Average', 'val_e_Average', 'atomicweight_Average', 'ionenergy_Average', 'el_neg_chi_Average', 'R_vdw_element_Average', 'R_cov_element_Average', 'zaratio_Average', 'density_Average']


,0,1
allelectrons_Total,100.000000,100.000000
density_Total,0.841611,7.558488
allelectrons_Average,10.000000,10.000000
val_e_Average,4.800000,4.800000
atomicweight_Average,20.612526,20.298893
ionenergy_Average,11.088100,12.040830
el_neg_chi_Average,2.766000,2.755000
R_vdw_element_Average,1.732000,1.631000
R_cov_element_Average,0.860000,0.910000
zaratio_Average,0.496070,0.492719


In [8]:
trainX = pd.DataFrame(columns=cols,data=scale(trainX0))
testX = pd.DataFrame(columns=cols,data=scale(testX0))

In [9]:
class Splitter:
    def __init__(self, n_splits=5, test_size=0.2):
        self.n_splits = n_splits
        self.test_size = test_size

    def split_data(self, X, y, random_state_list):
        for random_state in random_state_list:
            kf = KFold(n_splits=self.n_splits, random_state=random_state, shuffle=True)
            for train_index, val_index in kf.split(X, y):
                X_train, X_val = X.iloc[train_index], X.iloc[val_index]
                y_train, y_val = y.iloc[train_index], y.iloc[val_index]
                yield X_train, X_val, y_train, y_val

In [10]:
n_splits = 6
random_state = 42
random_state_list = [42] 
n_estimators = 999 
early_stopping_rounds = 333
verbose = False
device = 'cpu'
splitter = Splitter(n_splits=n_splits)

# Initialize an array for storing test predictions
test_predss = np.zeros((testX.shape[0],1))
ensemble_medae = []
weights = []
trained_models = {'xgb1':[], 'lgb1':[], 'cat1':[]}

In [11]:
import yaml

with open("/kaggle/input/pgs-s3e25-xgb1/Best_trial.yaml", "r") as yaml_file:
    Best_trial_xgb = yaml.safe_load(yaml_file)
with open("/kaggle/input/pgs-s3e25-lgbm1/Best_trial.yaml", "r") as yaml_file:
    Best_trial_lgb = yaml.safe_load(yaml_file)
with open("/kaggle/input/pgs-s3e25-catb1/Best_trial.yaml", "r") as yaml_file:
    Best_trial_cat = yaml.safe_load(yaml_file)

print(Best_trial_xgb)
print(Best_trial_lgb)
print(Best_trial_cat)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/pgs-s3e25-xgb1/Best_trial.yaml'

In [11]:
class Classifier:
    
    def __init__(self, n_estimators=1000, device="cpu", random_state=0):
        self.n_estimators = n_estimators
        self.device = device
        self.random_state = random_state
        self.models = self._define_model()
        self.models_name = list(self._define_model().keys())
        self.len_models = len(self.models)
        
    def _define_model(self):      
        xgb_optuna1 = Best_trial_xgb
        lgb_optuna1 = Best_trial_lgb
        cat_optuna1 = Best_trial_cat
    
        models = {
            'xgb1': xgb.XGBRegressor(**xgb_optuna1),
            'lgb1': lgb.LGBMRegressor(**lgb_optuna1),
            'cat1': CatBoostRegressor(**cat_optuna1),
        }
        
        return models

In [12]:
def median_absolute_error(y_true, y_pred):
    residuals = np.abs(y_true - y_pred)
    medae = np.median(residuals)
    return medae

In [13]:
classifier = Classifier(n_estimators, device, random_state)
models = classifier.models
display(models)

{'xgb1': XGBRegressor(alpha=1.2571313205216443, base_score=None, booster='gbtree',
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.42906980871128253, enable_categorical=False,
              eta=1.015938939074422, gamma=0.008210016182584622, gpu_id=None,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints=None, lambda=3.7837546445702372,
              learning_rate=0.1497327784963234, max_delta_step=None, max_depth=4,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              n_estimators=630, n_jobs=None, num_class=1, num_parallel_tree=None,
              predictor=None, random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=0.7296172169210334, ...),
 'lgb1': LGBMRegressor(alpha=0.5, bagging_fraction=0.5698024079281728, bagging_freq=4,
               colsample_bytree=0.5, feature_fraction=0.750866328122484,
               

In [14]:
oof_preds = []
test_preds = []
ensemble_medae = []
for i, (X_train_, X_val, y_train_, y_val) in enumerate(splitter.split_data(trainX, trainY, random_state_list=random_state_list)):
    n = i % n_splits
    m = i // n_splits
    
    # Get a set of Regressor models
    classifier = Classifier(n_estimators, device, random_state)
    models = classifier.models
    
    voting_model = VotingRegressor(estimators=[
        ('lgb1', models['lgb1']),
        ('xgb1', models['xgb1']),
        ('cat1', models['cat1'])
    ])    
    
    voting_model.fit(X_train_,y_train_)
    voting_model.score(X_val,y_val)
    
    test_pred =voting_model.predict(testX)
    y_val_pred = voting_model.predict(X_val)

    medae= median_absolute_error(y_val,y_val_pred)

    oof_preds.append(y_val_pred)
    test_preds.append(test_pred)
    ensemble_medae.append(medae)
    
    print('--------------')
    print(test_pred,test_pred.shape)
    print('--------------')


[LightGBM] [Warning] bagging_fraction is set=0.5698024079281728, subsample=0.9 will be ignored. Current value: bagging_fraction=0.5698024079281728
[LightGBM] [Warning] lambda_l1 is set=0.0010630752112872543, reg_alpha=0.1 will be ignored. Current value: lambda_l1=0.0010630752112872543
[LightGBM] [Warning] feature_fraction is set=0.750866328122484, colsample_bytree=0.5 will be ignored. Current value: feature_fraction=0.750866328122484
[LightGBM] [Warning] lambda_l2 is set=7.772527622708981, reg_lambda=1.0 will be ignored. Current value: lambda_l2=7.772527622708981
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=3 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] bagging_fraction is set=0.5698024079281728, subsample=0.9 will be ignored. Current value: bagging_fraction=0.5698024079281728
[LightGBM] [Warning] lambda_l1 is set=0.0010630752112872543, reg_alpha=0.1 will be ignored. Current value: lambda_l1=0.0010630752112872543
[LightGBM] [Warning] feature_fractio

In [15]:
# Calculate the mean LogLoss score of the ensemble
mean_score = np.mean(ensemble_medae)
std_score = np.std(ensemble_medae)
print(f'Ensemble MEDAE score {mean_score:.5f} ± {std_score:.5f}')

Ensemble MEDAE score 0.60107 ± 0.02923


In [16]:
print(np.array(test_preds).shape)
row_sums = np.mean(np.array(test_preds).T, axis=1)
print(row_sums)

(6, 6939)
[2.97661959 3.13770286 5.54409293 ... 4.77050717 4.24788953 2.81885799]


In [17]:
submit=pd.read_csv('/kaggle/input/playground-series-s3e25/sample_submission.csv')
submit['Hardness'] = row_sums
submit.to_csv('submission.csv',index=False)
display(submit)

,id,Hardness
0,10407,2.976620
1,10408,3.137703
2,10409,5.544093
3,10410,4.368978
4,10411,5.308063
...,...,...
6934,17341,5.770380
6935,17342,4.072489
6936,17343,4.770507
6937,17344,4.247890
